# Scale bar detection using an image processing algorithm

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from utils import detect_black_scale_bar

In [2]:
images_folder = "C:/Users/elyes/Documents/UNIL ants/EPFL/original/"

In [ ]:
from utils import resize_image
print(resize_image(images_folder + "anic32-900190-1_p_1.jpg", img_name="anic32-900190-1_p_1.jpg"))

In [ ]:
from utils import process_images_from_csv
process_images_from_csv("thorax_only.csv", "annotations.csv", images_folder)